In [206]:
import pandas as pd
import numpy as np
import time
from datetime import datetime as dt, date, timedelta
from polygon import RESTClient
from secret import API_KEY

In [207]:
# Getting API key and client
api_key = API_KEY
client = RESTClient(api_key)

In [208]:
# Importing buy/sell signals
data = pd.read_csv('signals.csv')

In [209]:
# Creating columns for prices purchased and sold at 
data['call_prices'] = np.zeros(len(data))
data['put_prices'] = np.zeros(len(data))

In [210]:
# Setting ticker
underlying_ticker = 'AAPL'

In [211]:
def get_option_ticker(timestamp, price, type):
    # Fixing timestamps
    date = pd.to_datetime(timestamp).date()
    date_more = date + timedelta(weeks=1)

    try:
        # Initialize a list to collect all contract data
        options_list = []

        # Fetch options data
        options_data = client.list_options_contracts(
            underlying_ticker=underlying_ticker,
            expiration_date_lte=date_more,
            expiration_date_gte=date,
            strike_price_gte=(price),
            strike_price_lte=(price+10),
            contract_type=type,
            limit=1000,
            expired=True
        )

        # Loop through the options data and append each contract's data to options_list
        for option in options_data:
            options_list.append(option.__dict__)

        # Convert the list of options data to a DataFrame
        df = pd.DataFrame(options_list)

        # Finding closest ticker
        option_ticker = df['ticker'].iloc[0]

        return option_ticker
    except Exception as e:
        print(f'Error: {e}')

    return None

def get_option_data(timestamp, ticker):
    date = pd.to_datetime(timestamp).date()

    try:
        aggs = client.get_aggs(
                ticker=ticker,
                multiplier=1,
                timespan="minute",
                from_=date,
                to=date,
                limit=5000
            )
        
        df = pd.DataFrame([agg.__dict__ for agg in aggs])

        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
        df['timestamp'] = df['timestamp'].dt.tz_localize('UTC').dt.tz_convert('America/New_York')

        df['time'] = df['timestamp'].dt.time
        df['time'] = df['time'].apply(lambda t: t.strftime("%H:%M:%S"))
        
        return df
    except Exception as e:
        print(f"Error: {e}")

    return None


In [214]:
buy_call_price = None
buy_put_price = None
sell_call_price = None
sell_put_price = None
option_price_data = None

for index, row in data.iterrows():
    if row['call_buy'] and row['call_prices'] == 0:
        timestamp = row['timestamp']
        time.sleep(12)
        ticker = get_option_ticker(timestamp, row['close'], 'call')
        time.sleep(12)
        option_price_data = get_option_data(timestamp, ticker)

        for i in range(0,3):
            try:
                row_time = (pd.to_datetime(row['time']) + timedelta(minutes=i)).strftime("%H:%M:%S")


                buy_call_price = (option_price_data[option_price_data['time'] == row_time])['close'].iloc[0]
                data.at[index, 'call_prices'] = buy_call_price
                
                print(index)
                break

            except Exception as e:
                print(f'No put price at: {timestamp}, {row_time}\nError: {e}')
                data.at[index, 'call_prices'] = None
                continue

    elif row['call_sell'] and row['call_prices'] == 0:
        for i in range(0,3):
            try:
                row_time = (pd.to_datetime(row['time']) + timedelta(minutes=i)).strftime("%H:%M:%S")


                sell_call_price = (option_price_data[option_price_data['time'] == row_time])['close'].iloc[0]
                data.at[index, 'call_prices'] = sell_call_price
                
                print(index)
                break

            except Exception as e:
                print(f'No put price at: {timestamp}, {row_time}\nError: {e}')
                data.at[index, 'call_prices'] = None
                continue

    elif row['put_buy'] and row['put_prices'] == 0:
        timestamp = row['timestamp']
        time.sleep(12)
        ticker = get_option_ticker(timestamp, row['close'], 'put')
        time.sleep(12)
        option_price_data = get_option_data(timestamp, ticker)

        for i in range(0,3):
            try:
                row_time = (pd.to_datetime(row['time']) + timedelta(minutes=i)).strftime("%H:%M:%S")


                buy_put_price = (option_price_data[option_price_data['time'] == row_time])['close'].iloc[0]
                data.at[index, 'put_prices'] = buy_put_price
                
                print(index)
                break
                
            except Exception as e:
                print(f'No put price at: {timestamp}, {row_time}\nError: {e}')
                data.at[index, 'put_prices'] = None
                continue

    elif row['put_sell'] and row['put_prices'] == 0:
        for i in range(0,3):
            try:
                row_time = (pd.to_datetime(row['time']) + timedelta(minutes=i)).strftime("%H:%M:%S")


                sell_put_price = (option_price_data[option_price_data['time'] == row_time])['close'].iloc[0]
                data.at[index, 'put_prices'] = sell_put_price
                
                print(index)
                break

            except Exception as e:
                print(f'No put price at: {timestamp}, {row_time}\nError: {e}')
                data.at[index, 'put_prices'] = None
                continue

658
659
No put price at: 2023-07-31 13:34:00+00:00, 13:34:00
Error: single positional indexer is out-of-bounds
660
661
No put price at: 2023-08-01 12:01:00+00:00, 12:01:00
Error: single positional indexer is out-of-bounds
662
663
664
665
No put price at: 2023-08-02 11:02:00+00:00, 11:02:00
Error: single positional indexer is out-of-bounds
666
667
668
669
670
671
672
673
674
675
676
677
678
No put price at: 2023-08-07 12:03:00+00:00, 12:11:00
Error: single positional indexer is out-of-bounds
679
680
681
682
683
684
685
686
687
688
689
690
691
692
693
694
695
696
697
698
699
No put price at: 2023-08-10 12:16:00+00:00, 12:16:00
Error: single positional indexer is out-of-bounds
700
701
702
703
704
705
706
707
708
709
710
711
712
713
714
715
716
717
718
719
No put price at: 2023-08-17 13:30:00+00:00, 13:30:00
Error: single positional indexer is out-of-bounds
No put price at: 2023-08-17 13:30:00+00:00, 13:31:00
Error: single positional indexer is out-of-bounds
No put price at: 2023-08-17 13:

In [215]:
data.to_csv('final_data.csv')

In [232]:
null_list = (data[(data.isna()['call_prices'] | data.isna()['put_prices']) == True].index).to_list()

for row_index in null_list:
    if row_index % 2 == 0:
        data = (data.drop([row_index, (row_index+1)])).copy()
        try:
            null_list.remove(row_index+1)
        except:
            continue
    else:
        data = (data.drop([(row_index -1), row_index])).copy()

In [240]:
data.to_csv('backtest_data.csv')